<a href="https://colab.research.google.com/github/LillyBal/PLS_SEM_Capstone/blob/main/CleaningDatasetPLSSEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning Data for PLS-SE Analysis


###Import data 
Import nummric csv file from Qualtrics & libraries pandas and numpy 


In [112]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/User+acceptability_May+21,+2022_08.58.csv')

###Cleaning

1. Drop irrelevant columns
2. Drop answers of testing phase (14 respondents)
3. Drop answers that not consent


In [113]:
data = data.drop(data.loc[:, 'StartDate':'Duration (in seconds)'].columns, axis = 1)
data = data.drop(data.loc[:, 'RecordedDate':'UserLanguage'].columns, axis = 1)

In [114]:
data = data.drop(data.loc[:15].index)

In [115]:
indexNames = data[  (data['Finished'] == 0) ].index
data.drop(indexNames , inplace=True)

###Create new Data frame
create new data frame and delete all demographical columns. Convert data points into numeric data type.

In [116]:
df = pd.DataFrame(data)
df = df.drop(df.loc[:, 'Intro':'Blockchain_Knowledge_11_TEXT'].columns, axis = 1)
df = df.drop(df.loc[:, 'Q14':'Q19 Occupation_6_TEXT'].columns, axis = 1)
df = df.drop('Q19 Mail', axis = 1)

df = df.apply(pd.to_numeric)

Add demographic variables.

In [117]:
df['Age'] = data['Q15_Age']
df['Gender'] = data['Q16 Gender']
df['Education'] = data['Q17 Education']
df['Occupation'] = data['Q19 Occupation']
df['Income'] = data['Q20 income']
df['Income_Descriptives'] = data['Q20 income']
df['Nationality'] = data['Q18 Nationality']
df['Blockchain_Knowledge'] = data['Blockchain_Knowledge']

In [118]:
df.describe()

,Finished,Q2 Technology_1,Q2 Technology_2,Q2 Technology_3,Q2 Technology_4,Q2 Technology_5,Q2 Technology_6,Q3 Voluntairness_1,Q3 Voluntairness_2,Q3 Voluntairness_3,...,Q11 Effort_3,Q12 Social Facil_1,Q12 Social Facil_2,Q12 Social Facil_3,Q12 Social Facil_4,Q12 Social Facil_5,Q13 Behaviour intent_1,Q13 Behaviour intent_2,Q13 Behaviour intent_3,Q20 income
count,337.000000,292.000000,290.000000,291.000000,290.000000,291.000000,290.000000,278.000000,278.000000,278.000000,...,187.000000,188.000000,188.000000,188.000000,188.000000,188.000000,187.000000,187.000000,187.000000,188.000000
mean,0.557864,3.229452,3.634483,4.336770,2.693103,3.061856,2.093103,3.964029,2.453237,3.679856,...,3.390374,2.771277,3.452128,3.265957,3.446809,3.202128,3.609626,3.320856,3.491979,3.021277
std,0.497379,1.123976,1.037790,0.963107,1.381647,1.303691,1.101252,0.960671,1.284301,0.977190,...,0.923128,0.984274,1.004187,1.091293,1.085806,1.197857,1.058764,1.012685,1.123402,1.402662
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.000000,3.000000,4.000000,1.000000,2.000000,1.000000,4.000000,1.000000,3.000000,...,3.000000,2.000000,3.000000,2.000000,3.000000,2.000000,3.000000,3.000000,3.000000,2.000000
50%,1.000000,3.000000,4.000000,5.000000,2.500000,3.000000,2.000000,4.000000,2.000000,4.000000,...,3.000000,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,3.000000
75%,1.000000,4.000000,4.000000,5.000000,4.000000,4.000000,3.000000,5.000000,4.000000,4.000000,...,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Missing values
Delete all rows with more than 5 missing values.

In [119]:
df = df[df.isnull().sum(axis=1) < 6]

Create a dataframe containing nationality and blockchain_knowledge since they have no nummeric data (e.g. Germany and Dutch 3,2) 

In [121]:
df2 = pd.DataFrame([df.Nationality, df.Blockchain_Knowledge]).transpose()
df2 = df2.reset_index(drop=True)
df2['Nationality'] = df2['Nationality'].fillna(0)
df = df.drop(df.loc[:, 'Nationality':'Blockchain_Knowledge'].columns, axis = 1)

Compute missing values with KNNImputer.

In [122]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(missing_values=np.nan, n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df),columns = df.columns)

Add Nationality and Blockchain_Knowledge to the data frame with the computed missing values.

In [123]:
df = df.join(df2, how='left')
df.rename(columns = {'Q12 Social Facil_1':'Q12 Social_1', 'Q12 Social Facil_2':'Q12 Social_2', 'Q12 Social Facil_3':'Q12 Facilitating_1', 'Q12 Social Facil_4':'Q12 Facilitating_2', 'Q12 Social Facil_5':'Q12 Facilitating_3'}, inplace = True)


#Download

Download new files for visualisation and analysis.

In [125]:
df.to_excel('data.xlsx')
df.to_csv('data.csv')